## Unspent Hasher PayOut (UHPO) Mechanism

In this NoteBook, we tried to implement all 3 types of Transactions from UHPO model.

Reference: https://gist.github.com/pool2win/77bb9b98f9f3b8c0f90963343c3c840f

![alt text](https://gist.githubusercontent.com/pool2win/77bb9b98f9f3b8c0f90963343c3c840f/raw/8fa2481728e7c12d553608af23ca6e551c7c90c1/uhpo-success.png "UHPO")



### Tx - Flow
- ALICE , BOB , CHARLIE , DAVID create their respective 1-4 blocks
- `payoutUpdate` is submitted on Time for 1-3 blocks after coinbase maturity
- `payoutUpdate` for coinbase 4 is not submitted on time, hence `payoutsettlement` is submitted to chain.
- After timelock expiry for coinbase 4 bob redeems timelock funds 

In [40]:
 

# reset regtest node
node :reset

# creating new key-pairs 
@alice = key :new 
@bob = key :new
@charlie = key :new
@david = key :new

# FROST keypairs for respective blocks
@poolkey_1 = key :new
@poolkey_2 = key :new
@poolkey_3 = key :new
@poolkey_4 = key :new

# miners can only spend funds after `x` blocks of coinbase maturity
@alice_timelock = "10 OP_CSV OP_DROP @alice OP_CHECKSIG"
@bob_timelock = "10 OP_CSV OP_DROP @bob OP_CHECKSIG"
@charlie_timelock = "10 OP_CSV OP_DROP @charlie OP_CHECKSIG" 
@david_timelock = "10 OP_CSV OP_DROP @david OP_CHECKSIG"


# coinbase outputs are taproot address where script_pub_key = FROST_POOL_PUBKEY + G*( MAST )
# where MAST has a depth of level 1 with only timelock contract
# funds can either be spent via frost-signatures or timelock script path

transition :build_braidpool_blocks do
    extend_chain taproot: { internal_key: @poolkey_1, leaves: [ @alice_timelock]}
    extend_chain taproot: { internal_key: @poolkey_2, leaves: [ @bob_timelock]}
    extend_chain taproot: { internal_key: @poolkey_3, leaves: [ @charlie_timelock]}
    extend_chain taproot: { internal_key: @poolkey_4, leaves: [ @david_timelock]}
end

mkdir -p /tmp/x &&              bitcoind -datadir=/tmp/x -chain=regtest              -rpcuser=test -rpcpassword=test -daemonwait -txindex -debug=1


#<Proc:0x00007f08c0c91970 (irb):28>

In [41]:

transition :payout_update_for_block_1 do
    extend_chain num_blocks: 97

    # Case blockheight is 101 (coinbase maturity for block 1) and payout update
    @poolkey_101 = key :new # using dummy keypair for now later will be replced with FROST group pubkey

    @poolkey_timelock_101 = key :new 
    @timelock_101 = "10 OP_CSV OP_DROP @poolkey_timelock_101 OP_CHECKSIG"
    
    @alice_coinbase_1 = get_coinbase_at 1
    
    @payout_update_101 = transaction inputs: [
                                    {
                                        tx: @alice_coinbase_1,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_1 } 
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_101, leaves: [ @timelock_101 ] }, # KP - We need leaves here. Don't we? In case the frost signature can't be created. 
                                                                                                # KP - Look at Payout Update transactions in the image above. There is one leaf with an OR condition.
                                           amount: 49.999.sats
                                         }
                                    ]
    
    broadcast @payout_update_101
    confirm transaction: @payout_update_101
end

#<Proc:0x00007f08bfdc7b10 (irb):1>

In [42]:
# payout update transaction which merges past payout funds and coinbase block 2 funds

transition :payout_update_for_block_2 do
    @bob_coinbase_2 = get_coinbase_at 2

    @poolkey_timelock_102 = key :new 
    @timelock_102 = "10 OP_CSV OP_DROP @poolkey_timelock_102 OP_CHECKSIG"

    @poolkey_102 = key :new 
    @payout_update_102 = transaction inputs: [
                                    {
                                        tx: @bob_coinbase_2,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_2 }
                                    }, { 
                                        tx: @payout_update_101,
                                         vout: 0,
                                         script_sig: { keypath: @poolkey_101 }, 
                                         sighash: :all
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_102, leaves: [ @timelock_102 ] }, 
                                           amount: 99.990.sats
                                         }
                                    ]
    
    broadcast @payout_update_102
    confirm transaction: @payout_update_102
end

#<Proc:0x00007f08bfe1c228 (irb):2>

In [43]:
# payout update transaction which merges past payout funds and coinbase block 3 funds

transition :payout_update_for_block_3 do
    @charlie_coinbase_3 = get_coinbase_at 3

    @poolkey_timelock_103 = key :new 
    @timelock_103 = "10 OP_CSV OP_DROP @poolkey_timelock_103 OP_CHECKSIG"

    @poolkey_103 = key :new 
    @payout_update_103 = transaction inputs: [
                                    {
                                        tx: @charlie_coinbase_3,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_3 }
                                    }, { 
                                        tx: @payout_update_102,
                                         vout: 0,
                                         script_sig: { keypath: @poolkey_102 }, 
                                         sighash: :all
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_103, leaves: [ @timelock_103 ] }, 
                                           amount: 149.980.sats
                                         }
                                    ]
    
    broadcast @payout_update_103
    confirm transaction: @payout_update_103
end

#<Proc:0x00007f08bfe11e18 (irb):2>

In [44]:
# case: payout-settlement (i.e; payoutUpdate for coinbase of 4th block is not submitted in time `10 blocks` )

transition :payout_settlement do
    extend_chain num_blocks: 10
    
    @payout_settlement_103 = transaction inputs: [
                                 { 
                                    tx: @payout_update_103,
                                     vout: 0,
                                     script_sig: { leaf_index: 0, sig: 'sig:@poolkey_timelock_103' }, 
                                     sighash: :all,
                                     csv: 10
                                }
                            ],
                            outputs: [
                                 { descriptor: 'wpkh(@alice)', amount: 30.sats },
                                 { descriptor: 'wpkh(@alice)', amount: 40.sats },
                                 { descriptor: 'wpkh(@alice)', amount: 30.sats },
                                 { descriptor: 'wpkh(@alice)', amount: 49.970.sats } 
                                ]

    broadcast @payout_settlement_103
    confirm transaction: @payout_settlement_103
end


#<Proc:0x00007f08bfe25f08 (irb):2>

In [45]:
# fail-case since payout update is not submitted in time bob can spend funds from coinbase 4 after 50 more blocks of maturity

# KP - Similar to this fail case we are missing the case when the frost signature completely fails. That is, more than threshold parties have been corrupted or are unreachable.

transition :david_spends_funds_after_timeout do
    extend_chain num_blocks: 10

    @david_coinbase_4 = get_coinbase_at 4
    @david_failure_close_tx = transaction inputs: [
                                    {
                                        tx: @david_coinbase_4,
                                        vout: 0,
                                        script_sig: { leaf_index: 0, sig: 'sig:@david' },
                                        csv: 10
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_101, leaves: [] }, 
                                           amount: 49.999.sats
                                         }
                                    ]
    
    broadcast @david_failure_close_tx
    confirm transaction: @david_failure_close_tx
end


#<Proc:0x00007f08bfadb9d8 (irb):4>

In [46]:
run_transitions :build_braidpool_blocks,
                :payout_update_for_block_1,
                :payout_update_for_block_2,
                :payout_update_for_block_3,
                :payout_settlement,
                :david_spends_funds_after_timeout

I, [2024-05-20T14:13:18.349434 #18]  INFO -- : Start build_braidpool_blocks
I, [2024-05-20T14:13:20.796201 #18]  INFO -- : Finish build_braidpool_blocks
I, [2024-05-20T14:13:20.796462 #18]  INFO -- : Start payout_update_for_block_1
I, [2024-05-20T14:13:22.524284 #18]  INFO -- : Finish payout_update_for_block_1
I, [2024-05-20T14:13:22.524531 #18]  INFO -- : Start payout_update_for_block_2
I, [2024-05-20T14:13:24.207948 #18]  INFO -- : Finish payout_update_for_block_2
I, [2024-05-20T14:13:24.208198 #18]  INFO -- : Start payout_update_for_block_3
I, [2024-05-20T14:13:25.804939 #18]  INFO -- : Finish payout_update_for_block_3
I, [2024-05-20T14:13:25.805224 #18]  INFO -- : Start payout_settlement
I, [2024-05-20T14:13:28.119088 #18]  INFO -- : Finish payout_settlement
I, [2024-05-20T14:13:28.120042 #18]  INFO -- : Start david_spends_funds_after_timeout
I, [2024-05-20T14:13:29.677666 #18]  INFO -- : Finish david_spends_funds_after_timeout


[:build_braidpool_blocks, :payout_update_for_block_1, :payout_update_for_block_2, :payout_update_for_block_3, :payout_settlement, :david_spends_funds_after_timeout]